In [71]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
from splinter import Browser
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options  
from time import sleep
import pymongo
import csv
import os   
from datetime import datetime, date
import dateutil.parser as dp
import numpy as np

def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path": "chromedriver.exe"}
    return Browser("chrome", **executable_path, headless=False)

def get_date(my_str):
    str_time = dp.parse(my_str).replace(tzinfo=None)
    return datetime.strftime(str_time,"%Y-%m-%d")

def feb_or_later(my_str):
    str_time = dp.parse(my_str).replace(tzinfo=None)
    start_time = dp.parse('2020-02-01T00:00:00.000Z').replace(tzinfo=None)
    return (str_time >= start_time)


def parse_tweet(tweet):
    entry = {}
            
    #Get Time of Tweet
    entry['time'] = tweet.find("time")['datetime']


    # Gets Text of Tweet
    # L is the "layers" of the tweet.  L[2] is the text itself.
    Layers = []
    Layers.append(tweet.findAll("div",recursive=False)[1])
    Layers.append(Layers[0].findAll("div",recursive=False)[1])
    Layers.append(Layers[1].findAll("div",recursive=False)[len(Layers[1])-3].text)
    entry['text'] = Layers[2]


    #Get Handle
    entry['handle'] = Layers[0].findAll('a')[0]['href']


    #Get Is_Retweet
    info = tweet.find_parent('div').find('div').text
    entry['is_retweet'] = ('retweet' in info.lower())
    
    return entry

In [72]:
dp.parse(get_date('2020-03-31T20:10:13.000Z'))

datetime.datetime(2020, 3, 31, 0, 0)

In [75]:
#Scrapes many tweets
def twitter(url):
    #SOURCE: https://medium.com/@dawranliou/twitter-scraper-tutorial-with-python-requests-beautifulsoup-and-selenium-part-2-b38d849b07fe
    chrome_options = Options()  
    chrome_options.add_argument("--headless") 
    driver = webdriver.Chrome(executable_path=os.path.abspath("chromedriver"), options=chrome_options)  
    driver.get(url)
    print('here')
    body = driver.find_element_by_tag_name('body')
    sleep(4)
    
    
    tweets = []
    last_tweet_date = date.today()
    run = True
    while run:
        body.send_keys(Keys.PAGE_DOWN)
        html = driver.page_source
        soup = bs(html,'lxml')
        timeline = soup.findAll("div", {"data-testid" : "tweet"})
        
        for tweet in timeline:
            try:
                response = parse_tweet(tweet)
                
                if not response['is_retweet']:
                    tweet_time = response['time']
                    run = feb_or_later(tweet_time)
                    
                    
                
                if run and (not (response in tweets)):
                    if not response['is_retweet']:
                        last_tweet_date = get_date(response['time'])
                    response['tweet_date'] = last_tweet_date
                    tweets.append(response)
                
                
            except TypeError:
                #This allows us to ignore deleted tweets
                pass


    driver.close()
    return tweets

    

    
    
 

#     links = set()

#     for i in range(num_tweets):
#         body.send_keys(Keys.PAGE_DOWN)
#         mars_twitter_html = driver.page_source
#         soup = bs(mars_twitter_html,'lxml')
#         current_lines = pd.Series([x['href'] for x in soup.find_all(class_='css-4rbku5 css-18t94o4 css-901oao r-1re7ezh r-1loqt21 r-1q142lx r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-3s2u2q r-qvutc0')])
#         links.update(current_lines)

#     filepath = 'links_scraped.html'
#     with open(filepath,'w') as text:
#         for line in links:
#             try:
#                 text.write(str(line))
#                 text.write('\n')
#             except UnicodeEncodeError:
#                 text.write('line skipped')
#                 text.write('\n')
    
    
    
#     filepath2= 'all_tweets.html'
#     with open(filepath2,'w') as text:
#         for line in links:
#             response = tweet(f'https://twitter.com{line}')
#             for char in response:
#                 try:
#                     text.write(char)
#                 except UnicodeEncodeError:
#                     pass
#             text.write('\n')
    
   

In [76]:
results = twitter('https://twitter.com/GovPritzker')


here


In [7]:
datetime.strftime(date.today(),"%Y-%b-%d")

'2020-Mar-31'

In [56]:
times = [dp.parse(x['time']) for x in results]

In [77]:
results

[{'time': '2020-03-31T20:10:13.000Z',
  'text': "Saving as many lives as possible is the one goal I will put above all others—every time.\n\nAs science guides our decisions, tomorrow I'll sign an executive order to extend our disaster proclamation, stay home order & suspension of on-site learning at schools thru the end of April.",
  'handle': '/GovPritzker',
  'is_retweet': False,
  'tweet_date': '2020-03-31'},
 {'time': '2020-03-31T20:25:56.000Z',
  'text': 'La propagación de este virus está creciendo, al igual que sus riesgos. \n\nNo nos podemos rendir.',
  'handle': '/GovPritzker',
  'is_retweet': False,
  'tweet_date': '2020-03-31'},
 {'time': '2020-03-31T20:25:56.000Z',
  'text': 'Todavía estamos dentro de nuestra capacidad, y estamos trabajando para adquirir nuevas ventilaciones para aumentar esa capacidad, pero de todos los modelos que hemos visto, nuestro mayor riesgo de golpear la capacidad no es ahora, pero semanas a partir de ahora.',
  'handle': '/GovPritzker',
  'is_retwe